In [ ]:
# Load the environment variables
from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
!mkdir -p tmp

In [ ]:
%%writefile ./tmp/agents-config.yaml
# agents-config.yaml
worker:
  name: Basic Worker
  instructions: |
    You are a single-step Worker. Execute the given task directly and return the best possible result.
    Do not ask clarifying questions. Do not handoff, delegate, or call tools unless explicitly provided.
    Prefer concise, correct outputs. Use Markdown formatting when helpful.
  model: gpt-4o-mini
  has_memory: True
  temperature: 0.2
  max_tokens: 1000

validator:
  name: Basic Validator
  instructions: |
    You are a Validator agent in a Worker–Validator loop. Your role is to **assess whether 
    the Worker’s output meets the task objective and success criteria**.
    
    Your job is **not to redo the research yourself**, but to:
    1. Decide if the output is acceptable for publication to senior business and technology leaders.
    2. If not, give precise, actionable feedback so the Worker can correct it.
  model: gpt-4o-mini
  temperature: 0.2
  max_tokens: 1000

In [ ]:
# Load the agents-config.yaml file

import json
import yaml

with open('./tmp/agents-config.yaml', 'r') as file:
    agent_config_data = yaml.safe_load(file)

formatted_json = json.dumps(agent_config_data, indent=4)
print(formatted_json)

In [ ]:
from datetime import datetime
from agents import Agent, ModelSettings, SQLiteSession

def create_agent(agent_type: str = None):
    """ 
    Creates and returns an Agent that matches the given definition in the agents-config 
    YAML file. Optionally returns a memory session if agent configuration calls for it.
    """

    if agent_type is None or not agent_type.strip():
        raise ValueError("agent_type must be a valid type of agent defined in agent-configs.yaml.")
    
    agent_config = agent_config_data.get(agent_type)
    if agent_config is None:
        raise ValueError(f"'{agent_type}' does not match an agent defined in agent-configs.yaml.")

    # Generate a timestamp string for unique naming
    now_string = datetime.now().strftime("%Y-%m-%dT%H:%M:%SU%s")

    # Build agent based on YAML specification
    try:
        agent_model_settings=ModelSettings(
            temperature=agent_config['temperature'],
            max_tokens=agent_config['max_tokens'],
        )

        agent_name = agent_config.get('name') or f"{agent_type}_{now_string}"

        new_agent = Agent(
            name=agent_name,
            instructions=agent_config['instructions'],
            model=agent_config['model'],
            model_settings=agent_model_settings
        )
    except:
        raise

    # Create memory session for agent if configured
    agent_has_memory = agent_config.get('has_memory') or False
    agent_session_name = f"{agent_name}__SESSION_{now_string}" if agent_has_memory else None
    agent_session = SQLiteSession(agent_session_name) if agent_session_name else None

    return (new_agent, agent_session)

worker, worker_sess = create_agent('worker')
validator, validator_sess = create_agent('validator')
